# Goal
The goal of this notebook is to optimize SmartCache using the Active Learning approach.

We define active learning as an approach when we execute the following loop:
1. Get suggested config from optimizer,
2. Apply suggested config to SmartCache,
3. Execute a fixed workload,
4. Collect the metrics from smart cache,
5. Register an observation with the optimizer.

This is different from online learning in that the workload is fixed. In an online learning situation the workload could fluctuate, and we would pick up the changing workload signatures. This is more complicated and will be our next step.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# TODO
Lots to do here.
<s>
1. Configure Workload
1. Loop:
    1. Configure SmartCache
    1. Execute Workload
    1. Collect Metrics
    1. Register/Suggest

We will do it in 2 steps:
1. Do it for a fixed workload without passing in any size/frequency estimators.
</s>
2. Add the estimators (will require upgrading the Optimizer to consume context features)

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import os
from subprocess import Popen, CREATE_NEW_CONSOLE
import sys

import grpc
import pandas as pd

In [ ]:
from mlos.Grpc.BayesianOptimizerFactory import BayesianOptimizerFactory
from mlos.Logger import create_logger

from mlos.Examples.SmartCache import HitRateMonitor, SmartCache, SmartCacheWorkloadGenerator, SmartCacheWorkloadLauncher
from mlos.Mlos.SDK import MlosExperiment
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Spaces import Point, SimpleHypergrid, ContinuousDimension

grpc_port = 50051
mlos_python_path = os.path.abspath(os.path.join(os.getcwd(), "..", "Mlos.Python"))

In [ ]:
logger = create_logger('Optimizing Smart Cache')
optimizer_service_grpc_channel = grpc.insecure_channel(f'localhost:{grpc_port}')
bayesian_optimizer_factory = BayesianOptimizerFactory(grpc_channel=optimizer_service_grpc_channel, logger=logger)

In [ ]:
# Optimization Problem
#
optimization_problem = OptimizationProblem(
    parameter_space=SmartCache.parameter_search_space,
    objective_space=SimpleHypergrid(name="objectives", dimensions=[ContinuousDimension(name="hit_rate", min=0, max=1)]),
    context_space=None,
    objectives=[Objective(name="hit_rate", minimize=False)]
)
optimizer = bayesian_optimizer_factory.create_remote_optimizer(optimization_problem=optimization_problem)

In [ ]:
import os
os.getpid()

In [ ]:

workload_launcher = SmartCacheWorkloadLauncher(logger=logger)
mlos_agent = workload_launcher.mlos_agent

# Let's configure the SmartCacheWorkload
#
mlos_agent.set_configuration(
    component_type=SmartCacheWorkloadGenerator,
    new_config_values=Point(
        workload_type='sequential_key_from_range',
        sequential_key_from_range_config=Point(
            min=0,
            range_width=2048
        )
    )
)

In [ ]:
# Now finally we build the experiment.
#
hit_rate_monitor = HitRateMonitor()
smart_cache_experiment = MlosExperiment(
    smart_component_types=[SmartCache],
    telemetry_aggregators=[hit_rate_monitor]
)
mlos_agent.start_experiment(smart_cache_experiment)

In [ ]:
num_iterations = 100
for i in range(num_iterations):
    mlos_agent.set_configuration(component_type=SmartCache, new_config_values=optimizer.suggest())
    hit_rate_monitor.reset()
    workload_launcher.start_workload(block=True)

    current_cache_config = mlos_agent.get_configuration(component_type=SmartCache)
    features_df = current_cache_config.to_pandas()
    hit_rate = hit_rate_monitor.get_hit_rate()
    objectives_df = pd.DataFrame({'hit_rate': [hit_rate]})
    optimizer.register(features_df, objectives_df)
    logger.info(f"[{i+1}/{num_iterations}]current_config: {current_cache_config}, hit_rate: {hit_rate}, num_requests: {hit_rate_monitor._num_requests}")

In [ ]:
hit_rate_monitor._num_requests

In [ ]:
# Clean up
#
mlos_agent.stop_experiment(smart_cache_experiment)
mlos_globals.mlos_global_context.stop_clock()
mlos_agent.stop_all()
